In [5]:
import numpy as np
import pandas as pd
from numpy import load
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, IterableDataset


In [6]:
#BCE Loss Test

x = torch.rand(1,10,dtype = float,requires_grad = True)
m = torch.rand(1,10,dtype = float)
y = torch.randint(2,(1,10),dtype = float)
z = x+m
g = nn.Sigmoid()

k = g(z)

loss = nn.BCELoss()
output = loss(k,y)

output.backward()


Loading Data

In [7]:
vanilla_data = '/Users/timothygould/dbg_research/research/training_data/vanilla_data/'
standard_norm = '/Users/timothygould/dbg_research/research/training_data/standard_norm/'
zero_to_one = '/Users/timothygould/dbg_research/research/training_data/zero_to_one_norm/'


def load_data(path):
    train_x=np.load(f'{path}train_x.npy',allow_pickle=True,fix_imports=True,encoding='latin1')
    train_y=np.load(f'{path}train_y.npy',allow_pickle=True,fix_imports=True,encoding='latin1')
    test_x=np.load(f'{path}test_x.npy',allow_pickle=True,fix_imports=True,encoding='latin1')
    test_y=np.load(f'{path}test_y.npy',allow_pickle=True,fix_imports=True,encoding='latin1')

    return train_x,train_y,test_x,test_y

In [8]:
train_x,train_y,test_x,test_y = load_data(standard_norm)
train_x = torch.from_numpy(train_x)
train_y = torch.from_numpy(train_y)
test_x = torch.from_numpy(test_x)
test_y = torch.from_numpy(test_y)

Creating Basic RNN Model

In [9]:
#Hyper Parameters

input_size = train_x.shape[0]
sequence_length = train_x.shape[2]
hidden_size = 24
num_classes = 2
num_epochs = 2
num_layers = 2
batch_size = train_x.shape[1]
learning_rate = 0.01
layers_dims = {'rnn':[hidden_size,hidden_size],'ff':[hidden_size,3,1]}

In [10]:
class model:
    
    def __init__(self,learning_rate,train_data,test_data,num_epochs,layers_dims):
        self.learning_rate = learning_rate
        self.train_data = train_data
        self.test_data = test_data
        self.num_epochs = num_epochs
        self.layers_dims = layers_dims
        

In [11]:
class model(model):
    
    def initialize_params(self):
        nx = self.train_data.shape[0]
        m = self.train_data.shape[1]
        tx = self.train_data.shape[2]

        self.ff_params = {}
        self.rnn_params = {}
        
        
        # RNN Params
        for i in range(0,len(layers_dims['rnn'])):
            self.rnn_params['a0'+str(i+1)] = torch.zeros(layers_dims['rnn'][0],m,dtype = float, requires_grad = True)
            self.rnn_params['Waa'+str(i+1)] = torch.rand(layers_dims['rnn'][i],layers_dims['rnn'][i],dtype = float, requires_grad = True)
            self.rnn_params['Wax'+str(i+1)] = torch.rand(layers_dims['rnn'][i],nx,dtype = float, requires_grad = True)
            self.rnn_params['Wya'+str(i+1)] = torch.rand(2,layers_dims['rnn'][i],dtype = float, requires_grad = True)
            self.rnn_params['ba'+str(i+1)] = torch.rand(layers_dims['rnn'][i],1,dtype = float, requires_grad = True)
            self.rnn_params['by'+str(i+1)] = torch.rand(layers_dims['rnn'][i],1,dtype = float, requires_grad = True)
        
        #FF Params
        for i in range(1,len(layers_dims['ff'])):
            self.ff_params['W'+str(i)] = torch.rand(layers_dims['ff'][i],layers_dims['ff'][i-1],dtype = float,requires_grad = True)
            self.ff_params['b'+str(i)] = torch.rand(layers_dims['ff'][i],1,dtype = float,requires_grad = True)
      

        return self.ff_params,self.rnn_params

In [12]:
test = model(learning_rate,train_x,train_y,num_epochs,layers_dims)
ff_params,rnn_params = test.initialize_params()
for k,v in rnn_params.items():
    print(k,v.shape)
for k,v in ff_params.items():
    print(k,v.shape)

a01 torch.Size([24, 1910])
Waa1 torch.Size([24, 24])
Wax1 torch.Size([24, 24])
Wya1 torch.Size([2, 24])
ba1 torch.Size([24, 1])
by1 torch.Size([24, 1])
a02 torch.Size([24, 1910])
Waa2 torch.Size([24, 24])
Wax2 torch.Size([24, 24])
Wya2 torch.Size([2, 24])
ba2 torch.Size([24, 1])
by2 torch.Size([24, 1])
W1 torch.Size([3, 24])
b1 torch.Size([3, 1])
W2 torch.Size([1, 3])
b2 torch.Size([1, 1])


In [13]:
class model(model):
    def ff_forward(self,a):
        
        for i in range(1,len(layers_dims['ff'])-1):
            w = ff_params['W'+str(i)]
            b = ff_params['b'+str(i)]
            #a value needs to be inherited from rnn cell output
            z = torch.matmul(w,a)+b
            a_next = torch.tanh(z)
        
        w = ff_params['W'+str(len(layers_dims['ff'])-1)]
        b = ff_params['b'+str(len(layers_dims['ff'])-1)]
        z = torch.matmul(w,a_next)+b
        y_pred = torch.sigmoid(z)
        
        return y_pred

        

In [14]:
test = model(learning_rate,train_x,train_y,num_epochs,layers_dims)
ff_params,rnn_params = test.initialize_params()
a = torch.rand(24,10,dtype = float)


test.ff_forward(a)

tensor([[0.8235, 0.8235, 0.8235, 0.8235, 0.8235, 0.8235, 0.8235, 0.8235, 0.8235,
         0.8235]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [15]:
class model(model):
    
    def rnn_cell_forward(self,layer,a_prev,xt):
        Wax = rnn_params['Wax'+str(layer)]
        Waa = rnn_params['Waa'+str(layer)]
        ba = rnn_params['ba'+str(layer)]
        z = torch.matmul(Wax,xt)+torch.matmul(Waa,a_prev)+ba
        a_next = torch.tanh(z)
        
        return a_next

In [16]:
test = model(learning_rate,train_x,train_y,num_epochs,layers_dims)
ff_params,rnn_params = test.initialize_params()
xt = train_x[:,:,23]
a_prev = rnn_params['a01']
layer = 1
test.rnn_cell_forward(layer,a_prev,xt)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], dtype=torch.float64,
       grad_fn=<TanhBackward>)

In [17]:
class model(model):
    
    def forward_pass(self):
        
        tx = self.train_data.shape[2]
        
        a_next1 = rnn_params['a01']
        a_next2 = rnn_params['a02']
        for xt in range(tx):
            a_next1 =  self.rnn_cell_forward(1,a_next1,train_x[:,:,xt])
            #Wax cannot be multipied by a_next1 for 2nd layer 24x24 * 10x1910
            #solution, square matricies. Hidden layers matches nx
           
            a_next2 = self.rnn_cell_forward(2,a_next2,a_next1)
            #return values for ff layers to run through
        y_pred = self.ff_forward(a_next2)
        print(y_pred)
        
        #return y_pred = ff prop (takes in 2nd a value as x val)

In [18]:
test = model(learning_rate,train_x,train_y,num_epochs,layers_dims)
ff_params,rnn_params = test.initialize_params()
test.forward_pass()




tensor([[0.8045, 0.8045, 0.8045,  ..., 0.8045, 0.8045, 0.8045]],
       dtype=torch.float64, grad_fn=<SigmoidBackward>)


In [19]:
w = torch.rand(5,3)
print(w)

tensor([[0.8127, 0.5111, 0.9062],
        [0.7860, 0.5888, 0.3492],
        [0.4600, 0.4561, 0.8919],
        [0.7765, 0.7589, 0.1656],
        [0.5601, 0.4765, 0.9060]])


In [20]:
w = nn.init.xavier_uniform_(w)
print(w)

tensor([[ 0.7856,  0.1435, -0.7904],
        [ 0.5704,  0.2707, -0.4248],
        [-0.2174,  0.7737,  0.5865],
        [ 0.1716,  0.4342, -0.0140],
        [ 0.4336, -0.5035,  0.4222]])


In [21]:
params = {'w1':torch.rand(3,4),'w2':torch.rand(8,5)}
print(params)

{'w1': tensor([[0.6949, 0.1975, 0.2390, 0.2042],
        [0.1335, 0.9050, 0.0642, 0.4983],
        [0.0484, 0.2332, 0.4376, 0.1348]]), 'w2': tensor([[0.7446, 0.5333, 0.5192, 0.7687, 0.9752],
        [0.2183, 0.7866, 0.2300, 0.0522, 0.1432],
        [0.5145, 0.1058, 0.0573, 0.5333, 0.5345],
        [0.0561, 0.5997, 0.7050, 0.8247, 0.6537],
        [0.3568, 0.9738, 0.4067, 0.4280, 0.0692],
        [0.1662, 0.4695, 0.0708, 0.0576, 0.6663],
        [0.9076, 0.2296, 0.1046, 0.7135, 0.9525],
        [0.7808, 0.9877, 0.0641, 0.3036, 0.8319]])}


In [22]:
params['w1'] = nn.init.xavier_uniform_(params['w1'])

In [23]:
print(params)

{'w1': tensor([[-0.6140, -0.6079,  0.1809, -0.2050],
        [ 0.0531,  0.5679,  0.5856,  0.4941],
        [-0.1231, -0.5918, -0.6656, -0.4224]]), 'w2': tensor([[0.7446, 0.5333, 0.5192, 0.7687, 0.9752],
        [0.2183, 0.7866, 0.2300, 0.0522, 0.1432],
        [0.5145, 0.1058, 0.0573, 0.5333, 0.5345],
        [0.0561, 0.5997, 0.7050, 0.8247, 0.6537],
        [0.3568, 0.9738, 0.4067, 0.4280, 0.0692],
        [0.1662, 0.4695, 0.0708, 0.0576, 0.6663],
        [0.9076, 0.2296, 0.1046, 0.7135, 0.9525],
        [0.7808, 0.9877, 0.0641, 0.3036, 0.8319]])}


In [24]:
for k,v in params.items():
    print(params[k],params[k].shape)
    params[k] = nn.init.xavier_uniform_(params[k])
    print(params[k],params[k].shape)

tensor([[-0.6140, -0.6079,  0.1809, -0.2050],
        [ 0.0531,  0.5679,  0.5856,  0.4941],
        [-0.1231, -0.5918, -0.6656, -0.4224]]) torch.Size([3, 4])
tensor([[ 0.4487,  0.3617, -0.6326, -0.7264],
        [ 0.5749, -0.8157,  0.1597, -0.7433],
        [-0.0945,  0.4967, -0.1029,  0.7560]]) torch.Size([3, 4])
tensor([[0.7446, 0.5333, 0.5192, 0.7687, 0.9752],
        [0.2183, 0.7866, 0.2300, 0.0522, 0.1432],
        [0.5145, 0.1058, 0.0573, 0.5333, 0.5345],
        [0.0561, 0.5997, 0.7050, 0.8247, 0.6537],
        [0.3568, 0.9738, 0.4067, 0.4280, 0.0692],
        [0.1662, 0.4695, 0.0708, 0.0576, 0.6663],
        [0.9076, 0.2296, 0.1046, 0.7135, 0.9525],
        [0.7808, 0.9877, 0.0641, 0.3036, 0.8319]]) torch.Size([8, 5])
tensor([[ 0.4147, -0.3972, -0.2341,  0.6169, -0.4231],
        [-0.6341, -0.0423, -0.4238, -0.0805,  0.0355],
        [ 0.4886,  0.4583,  0.2248,  0.0619,  0.2203],
        [-0.2193, -0.4814,  0.1379,  0.1414,  0.2253],
        [-0.4011, -0.0355,  0.5021, -0.5076,

##### Experimenting with vectorized false positive and accuracy implementations

In [25]:
y_pred = torch.rand(1,24)
y = torch.rand(1,24)
y = torch.bernoulli(y)
print(y)
print(y_pred)
print(type(y_pred))

tensor([[1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
         1., 1., 0., 1., 1., 1.]])
tensor([[0.0092, 0.6231, 0.5359, 0.3593, 0.4444, 0.2502, 0.0843, 0.3146, 0.6527,
         0.7774, 0.0716, 0.1799, 0.1980, 0.0622, 0.1765, 0.7221, 0.5847, 0.7088,
         0.9482, 0.1248, 0.4708, 0.6842, 0.6301, 0.3534]])
<class 'torch.Tensor'>


In [26]:
print(np.round(y_pred,1))

y_pred = torch.where(np.round(y_pred,1)==.7,1,0)
print(y_pred)

tensor([[0.0000, 0.6000, 0.5000, 0.4000, 0.4000, 0.3000, 0.1000, 0.3000, 0.7000,
         0.8000, 0.1000, 0.2000, 0.2000, 0.1000, 0.2000, 0.7000, 0.6000, 0.7000,
         0.9000, 0.1000, 0.5000, 0.7000, 0.6000, 0.4000]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]])


In [27]:
#checking for false positive
test = np.where((y==0) & (y_pred==1),1,0)
print(test)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]


In [28]:
test2 = np.sum(test,axis=1)
print(test2/24)

[0.04166667]


In [29]:
#checking for true positive
test3 = np.where((y==1) & (y_pred ==1),1,0)
print(test3)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]]
